# Sea Surface Temperature

In [ ]:
# import pickle
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from maps import map_robinson
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl
from regions import Nino12, Nino34
from plotting import shifted_color_map
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA
from xr_regression import xr_linear_trends_2D, xr_linear_trend, ocn_field_regression

# global SST map and trends

In [ ]:
MASK = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).REGION_MASK
TAREA = xr_AREA('ocn')

In [ ]:
# files produced in SST.ipynb
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc')
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_rcp.nc' )

In [ ]:
# Niño 3.4 (5N-5S, 170W-120W)
print(find_array_idx(SST_yrly_ctrl.TLAT[:,0], -5).item())
print(find_array_idx(SST_yrly_ctrl.TLAT[:,0], +5).item())
print(find_array_idx(SST_yrly_ctrl.TLONG[500,:], 360-170).item())
print(find_array_idx(SST_yrly_ctrl.TLONG[500,:], 360-120).item())

# Niño 1+2 (0-10S, 90W-80W)
print(find_array_idx(SST_yrly_ctrl.TLAT[:,0], -10).item())
print(find_array_idx(SST_yrly_ctrl.TLAT[:,0], 0  ).item())
print(find_array_idx(SST_yrly_ctrl.TLONG[500,:], 360-90).item())
print(find_array_idx(SST_yrly_ctrl.TLONG[500,:], 360-80).item())

In [ ]:
N12_ctrl = ((SST_yrly_ctrl*TAREA).sel(Nino12).sum(dim=('nlat','nlon'))/TAREA.sel(Nino12).sum(dim=('nlat','nlon')))
N34_ctrl = ((SST_yrly_ctrl*TAREA).sel(Nino34).sum(dim=('nlat','nlon'))/TAREA.sel(Nino34).sum(dim=('nlat','nlon')))
N12_rcp  = ((SST_yrly_rcp *TAREA).sel(Nino12).sum(dim=('nlat','nlon'))/TAREA.sel(Nino12).sum(dim=('nlat','nlon')))
N34_rcp  = ((SST_yrly_rcp *TAREA).sel(Nino34).sum(dim=('nlat','nlon'))/TAREA.sel(Nino34).sum(dim=('nlat','nlon')))

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)

plt.plot(N12_ctrl.values, c='C0', lw=2, ls='-' , label='CTRL $Ni\~no$ 1+2')
plt.plot(N34_ctrl.values, c='C0', lw=2, ls='--', label='CTRL $Ni\~no$ 3.4')
plt.plot(N12_rcp.values , c='C1', lw=2, ls='-' , label='RCP $Ni\~no$ 1+2')
plt.plot(N34_rcp.values , c='C1', lw=2, ls='--', label='RCP $Ni\~no$ 3.4')

plt.xlabel('time [years]', fontsize=16)
plt.ylabel('raw ENSO indices [$^\circ$C]', fontsize=16)
plt.legend(fontsize=16, ncol=2)
plt.savefig(f'{path_results}/SST/ENSO_indices')